In [2]:
!pip freeze | grep scikit-learn

scikit-learn==1.6.1


In [3]:
!python -V

Python 3.11.13


In [4]:
import pickle
import pandas as pd

In [8]:
with open('model.bin', 'rb') as f_in:
    dv, model = pickle.load(f_in)

/usr/local/lib/python3.11/dist-packages/sklearn/base.py:380: InconsistentVersionWarning: Trying to unpickle estimator DictVectorizer from version 1.5.0 when using version 1.6.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/base.py:380: InconsistentVersionWarning: Trying to unpickle estimator LinearRegression from version 1.5.0 when using version 1.6.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


In [9]:
categorical = ['PULocationID', 'DOLocationID']

def read_data(filename):
    df = pd.read_parquet(filename)

    df['duration'] = df.tpep_dropoff_datetime - df.tpep_pickup_datetime
    df['duration'] = df.duration.dt.total_seconds() / 60

    df = df[(df.duration >= 1) & (df.duration <= 60)].copy()

    df[categorical] = df[categorical].fillna(-1).astype('int').astype('str')

    return df

In [10]:
df = read_data('https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-03.parquet')

In [11]:
dicts = df[categorical].to_dict(orient='records')
X_val = dv.transform(dicts)
y_pred = model.predict(X_val)

**Que 1** What's the standard deviation of the predicted duration?

In [12]:
print(f'standard deviation of predictions: {y_pred.std():.2f}')

standard deviation of predictions: 6.25


In [13]:
year = 2023
month = 3
df['ride_id'] = f'{year:04d}/{month:02d}_' + df.index.astype('str')

In [14]:
df_result = pd.DataFrame()

In [15]:
df_result['ride_id'] = df['ride_id']
df_result['prediction'] = y_pred

In [16]:
df_result.to_parquet(
    'output_file',
    engine='pyarrow',
    compression=None,
    index=False
)

**Que 2** What is the size of output file?

In [17]:
size = !stat -c %s $'/content/output_file'

In [18]:
print(f'size of output file: {int(size[0])/1048576:.2f} MB')

size of output file: 65.46 MB


In [19]:
!jupyter nbconvert --to script starter.ipynb

[NbConvertApp] Converting notebook starter.ipynb to script
[NbConvertApp] Writing 1702 bytes to starter.py
